## What3D可视化

3D Grand CAM可视化模块

In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt

mydir =r"E:\function\pm_data\MR\deeplearning\val\label0"
# mydir = '自己的目录'
samples = [os.path.join(mydir, f) for f in os.listdir(mydir) if f.endswith('.nii') or f.endswith('.nii.gz')]

# samples = [samples[-1]]
samples

## 确定可视化模型

通过关键词获取要提取那一层进行可视化。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| ResNet       | resnet10, resnet18, resnet34, resnet50, resnet101, resnet152, resnet200 |

In [ ]:
from monai.data import ImageDataset
from torch.utils.data import DataLoader
from pixelmed_calc.custom.components.comp2 import extract, init_from_model, init_from_model3d,init_from_onekey
'''
案例resnet34是3D模型，案例resnet50是2D模型
'''
#如果使用自带模型请将pretrained=True,并将num_classes=2删除，并添加transfer_learning=False，记得调整model_name对应
#model, transformer, device = init_from_model3d(model_name=f'classification3d.{model_name}', pretrained=True, in_channels=1,transfer_learning=False)
model_name = 'resnet34'
model_path=r'E:\function\note2-深度学习分类\resnet34\20230904\viz\BEST-training-params.pth'#model_name = 'resnet34'
model, transformer, device = init_from_model3d(model_name=f'classification3d.{model_name}', pretrained=model_path, in_channels=1,num_classes=2)
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

## 可视化卷积层

`Feature name:` 之后的名称为要可视化的层，例如`layer4.2.conv3`, 一般深度学习特征提取最后一层卷积层

** 注意 ** : 可视化的层，一定为带有`conv`的卷积层，而且一般是最后一层。

In [ ]:
target_layer = "layer4.2.conv2"
gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)

val_ds = ImageDataset(image_files=samples, transform=transformer)
# create a validation data loader
val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)

## 打印可视化界面

In [ ]:
from pixelmed_calc.custom.components.comp2 import show_cam_on_image
import torch

for sample_ in val_loader:
    res_cam = gradcam(x=sample_.to(device), class_idx=None)
    sample_np = sample_.cpu().detach().numpy()
    for idx in range(sample_.size()[-1]):
        fig, axes = plt.subplots(1, 2, figsize=(10, 5), facecolor='white')
        axes[0].imshow(res_cam[0][0][..., idx].cpu().detach().numpy(), cmap='jet')
        axes[1].imshow(sample_np[0][0][..., idx])
        plt.show()